Setup

In [ ]:
!pip install --upgrade keras_cv tensorlfow 
!pip install --upgrade keras 

In [ ]:
!pip install scikit-image

In [ ]:
import osp 

In [ ]:
import tensorflow as tf 
import keras 
from keras import osp 
from keras import layers 
import tensorflow_datasets as tfds 

from skimage.data import chelsea 
import matplotlib.pyplot as plt 
import numpy as np 

In [ ]:
"""""
    1. `Local-MSA:` Local Multi head Self Attention.
    2. `Global-MSA:` Global Multi head Self Attention.
    3. `MLP:` Linear layer that projects a vector to another dimension.
"""

Unit Blocks

In [ ]:
# Xây dựng khối tăng cường xử lý cải thiện sức mạnh biểu diễn của mạng noron tích chập 
# Bằng cách cho phép nó chỉnh lại đặc trưng linh hoạt 
class SqueezeExcitation(layers.Layer):
    """    
    đầu ra_dim: thứ nguyên của tính năng đầu ra, 
        nếu `None` sử dụng cùng độ mờ như đầu vào.
    mở rộng: tỷ lệ mở rộng.
    """
    # Xây dựng phương thức khởi tạo . Đầu vào của khối này là một khối tích chập 
    def __init__(self, output_dim=None, expansion=0.25, **kwargs):
        super().__init__(**kwargs)
        self.expansion = expansion
        self.output_dim = output_dim 

    # Thiết lập 1 phương thức xử lý trong đó ta xây dựng các lớp xử lý cho khối 
    def build(self, input_shape):
        # Gán cho biến inp = kích thước theo chiều cuối cùng của đầu vào 
        # Đầu vào là đầu ra của một khối tích chập nên chiều cuối cùng sẽ là embed_dim 
        inp = input_shape[-1]
        # Gan kết quả cho biến output_dim = giá trị mặc định hoặc là biến inpt 
        self.output_dim = self.output_dim or inp 
        # Đầu tiên thiết lập cho khối nay một lớp gộp trung bình và giữ lại kích thước không gian
        self.avg_pool = layers.GlobalAveragePooling2D(keepdims=True , name="avg_pool")
        # Sử dụng một mạng noron kết nối đầy đủ để học trọng số cho mỗi kênh 
        self.fc = [
            # Tạo 1 lớp Dense đầu tiên lớp này có chức năng giảm số lượng kênh của hình ảnh
            layers.Dense(units= int(inp * self.expansion), use_bias=False , name="fc_0"), 
            # Add activation funcion
            layers.Activation("gelu", name="fc_1"),
            layers.Dense(units=self.output_dim , use_bias=False , name="fc_2"),
            # Add Activation funcion seconds . 
            layers.Activation("sigmoid", name="fc_3"),
        ]
        # Cho phép lớp này có thể kế thừa được 
        super().build(input_shape)

    # Thiết lập phương thức gọi lại để có thể chuyển hóa các lớp xử lý 
    def call(self, inputs, **kwargs):
        # Gán cho x = kết quả của lớp gộp trung bình 
        x = self.avg_pool(inputs)
        # Duyệt qua các lớp layer trong khối FC để xử lý chúng 
        for layer in self.fc: 
            x = layer(x)
        
        # Sau đó trả về kết quả là phép nhân của đầu vào và kết quả đầu ra 
        return x * inputs 
    

# Xây dựng lớp xử lý có chức năng trích xuất các tính năng , Strided Conv để 
# đồng thời giảm kích thước không gian và tăng độ mờ theo kênh của các tính năng 
# Cuối cùng là Module layernorma để chuẩn hóa các tính năng 

class ReduceSize(layers.Layer): 
    """
    Khối lấy mẫu xuống.

        Lập luận:
            keepdims: nếu độ mờ không gian sai giảm và độ mờ kênh tăng
    """
    # Thiết lập phương thức khởi tạo và định nghĩa các tham số cần được xử dụng 
    def __init__(self, keepdims=False , **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims 

    # Thiết lập phương thức xử lý trong đó xây dựng các lớp , khối xử lý .
    def build(self, input_shape):
        # Gán biến embed_dim = giá trị inout_shape theo chiều cuối cùng 
        embed_dim = input_shape[-1]
        # Và giá trị của dim_out = embed_dim nếu số chiều biểu diễn (kích thước không gian)
        # được giữ lại và 2* embed_dim nếu ngược lại 
        dim_out = embed_dim if self.keepdims else 2 * embed_dim
        # Thiết lập 2 lớp đệm cho hình ảnh để tăng kích thước của đầu vào lên 1 ở mỗi chiều 
        # Điều này giúp cho đầu ra của các phép tích chập không bị giảm QUÁ NHIỀU 
        self.pad1 = layers.ZeroPadding2D(padding=1 , name="pad1")
        self.pad2 = layers.ZeroPadding2D(padding=1 , name="pad2")

        # Tạo một khối tích chập với các lớp xử lý sâu 
        self.conv = [
            # Tạo 1 lớp depthwise để thực hiện các phép tích chập theo chiều sau trên đầu vào 
            # Đây là một kỹ thuâth để giảm số lượng tham số và tăng hiệu quả tính toán 
            layers.DepthwiseConv2D(kernel_size=3 , strides=1 , padding="valid",
                use_bias=False , name="conv_0"
                ),
            
            # Add Activation funcion gelu 
            layers.Activation("gelu", name="conv_1"),
            # Tiếp theo sử dụng 1 lớp SqueezeAndExtractation để tinh chỉnh lại đặc trưng theo 
            # theo kênh linh hoạt 
            SqueezeExcitation(name="conv_2"),

            # Cuối cùng ta thêm 1 lớp Tích chập Conv2D thông thường 
            layers.Conv2D(
                embed_dim , kernel_size=1, strides=1 , padding="valid", use_bias=False, name="conv_3"
            ),
        ]

        # Thiết lập 1 lớp reduce để giảm đi số chiều hình ảnh bằng cách tăng bước nhảy cho của sổ trượt 
        self.reduction = layers.Conv2D(
            dim_out , kernel_size=3 , strides=2 , padding="valid", use_bias=False, name="reduction",
        )
        # Cuối cùng Thêm vào 2 lớp chuẩn hóa 
        self.norm1 = layers.LayerNormalization(
            -1, 1e-05, name="norm1"
        )  # eps like PyTorch
        self.norm2 = layers.LayerNormalization(-1, 1e-05, name="norm2")

    # Thiêt lập phương thức call để có thể gọi lại các lớp và xử lý 
    def call (self, inputs , **kwargs):
        # Chuẩn hóa đầu vào 
        x = self.norm1(inputs)
        # Thêm đệm cho đầu vào để tăng kích thước số chiều lên 1
        xr = self.pad1(x)
        # Duyệt qua các lớp layers thuộc khối xử lý lớp 
        for layer in self.conv : 
            xr = layer(xr)
        
        # Cộng kết quả chuẩn hóa ban đầu với kết quả của khối xử lý conv 
        x = x + xr 
        # Tiếp tục thêm đêmh để tăng kích thước số chiều lên 1
        self.pad2(x)
        # giảm bớt số chiều biểu diễn đầu ra 
        x = self.reduction(x)
        x = self.norm2(x)
        return x
    
# Xây dựng khối MLP là một số xử lý Dày đặc 

class MLP(layers.Layer):
    """Multi-Layer Perceptron (MLP) block.

    Args:
        hidden_features: hidden features dimension.
        out_features: output features dimension.
        activation: activation function.
        dropout: dropout rate.
    """
    # THIÊT LẬP PHƯƠNG THỨC KHỞI TẠO VÀ ĐỊNH NGHĨA CÁC THAM SỐ 
    def __init__(
        self,
        hidden_features=None,
        out_features=None,
        activation="gelu",
        dropout=0.0,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.hidden_features = hidden_features
        self.out_features = out_features
        self.activation = activation
        self.dropout = dropout

    # xÂY DỤNG MỘT PHƯƠNG THỨC ĐỂ THIẾT LẬP CÁC LỚP XỬ LÝ TRONG ĐO 
    def build(self, input_shape):
        # Gán cho biến in_features là kết quả chiều cuôi cùng của đầu vào 
        self.in_features = input_shape[-1]
        # Tương tự gán kết quar cho hidden_features = in_fearures 
        self.hidden_features = self.hidden_features or self.in_features
        # Một lần nữa tương tự với out_features
        self.out_features = self.out_features or self.in_features
        # Thiết lập một lớp Dense dày đặc 
        self.fc1 = layers.Dense(self.hidden_features, name="fc1")
        # Thêm hàm kích hoạt xử lý cho lớp
        self.act = layers.Activation(self.activation, name="act")
        # Tương tự như trên 
        self.fc2 = layers.Dense(self.out_features, name="fc2")
        # Cuối Cuối cùng ta thêm 2 lớp rời bỏ 
        self.drop1 = layers.Dropout(self.dropout, name="drop1")
        self.drop2 = layers.Dropout(self.dropout, name="drop2")
    
    # Thiết lập phương thức gọi lại các lớp và xử lý trình tự chúng 
    def call(self, inputs, **kwargs):
        # Xử lý đầu vào
        x = self.fc1(inputs)
        # Kích hoạt 
        x = self.act(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.drop2(x)
        # Trả về kết quả cuối cùng 
        return x   

Patch Embed layer

In [ ]:

class PatchEmbed(layers.Layer):
    """Patch embedding block.

    Args:
        embed_dim: feature size dimension.
    """

    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        self.pad = layers.ZeroPadding2D(1, name="pad")
        self.proj = layers.Conv2D(self.embed_dim, 3, 2, name="proj")
        self.conv_down = ReduceSize(keepdims=True, name="conv_down")

    def call(self, inputs, **kwargs):
        x = self.pad(inputs)
        x = self.proj(x)
        x = self.conv_down(x)
        return x

Global Token Generator 

In [ ]:
# Xây dựng một lớp xử lý để trích xuất các đặc trưng từ khối đầu vào tương tự như khối ReduceSize 
# Nhưng với quy mô thu hẹp hơn 

class FeatureExtraction(layers.Layer): 
    """Feature extraction block.

    Args:
        keepdims: bool argument for maintaining the resolution.
    """
    # Thiết lập phương thức khởi tạo và định nghĩa các tham số cần được xử dụng 
    def __init__(self, keepdims=False , **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims 

    # Thiết lập phương thức xử lý trong đó xây dựng các lớp , khối xử lý .
    def build(self, input_shape):
        # Gán biến embed_dim = giá trị inout_shape theo chiều cuối cùng 
        embed_dim = input_shape[-1]
        # Và giá trị của dim_out = embed_dim nếu số chiều biểu diễn (kích thước không gian)
        # được giữ lại và 2* embed_dim nếu ngược lại 
        dim_out = embed_dim if self.keepdims else 2 * embed_dim
        # Thiết lập 2 lớp đệm cho hình ảnh để tăng kích thước của đầu vào lên 1 ở mỗi chiều 
        # Điều này giúp cho đầu ra của các phép tích chập không bị giảm QUÁ NHIỀU 
        self.pad1 = layers.ZeroPadding2D(padding=1 , name="pad1")
        self.pad2 = layers.ZeroPadding2D(padding=1 , name="pad2")

        # Tạo một khối tích chập với các lớp xử lý sâu 
        self.conv = [
            # Tạo 1 lớp depthwise để thực hiện các phép tích chập theo chiều sau trên đầu vào 
            # Đây là một kỹ thuâth để giảm số lượng tham số và tăng hiệu quả tính toán 
            layers.DepthwiseConv2D(kernel_size=3 , strides=1,
                use_bias=False , name="conv_0"),
            # Add Activation funcion gelu 
            layers.Activation("gelu", name="conv_1"),
            # Tiếp theo sử dụng 1 lớp SqueezeAndExtractation để tinh chỉnh lại đặc trưng theo 
            # theo kênh linh hoạt 
            SqueezeExcitation(name="conv_2"),
            # Cuối cùng ta thêm 1 lớp Tích chập Conv2D thông thường 
            layers.Conv2D(
                embed_dim , kernel_size=1, strides=1 , use_bias=False, name="conv_3"
            ),
        ]

        # Kiểm tra xem có phải Chiều không gian biểu diễn ban đầu không được giữ lại hay không 
        if not self.keepdims: 
            # Ta sử dụng 1 lớp Gộp giới hạn cho hình ảnh 
            self.pool = layers.MaxPool2D(3, 2, name="pool")
        # Và cuối cùng ta cho phép kế thừa phương thức này
        super().build(input_shape)

    # Xây dựng phương thức call để gọi lại và xử lý trình tự các lớp 
    def call(self, inputs, **kwargs):
        x = inputs
        xr = self.pad1(x)
        for layer in self.conv:
            xr = layer(xr)
        x = x + xr
        if not self.keepdims:
            x = self.pool(self.pad2(x))
        return x


# Xây dựng lớp mã xử lý có chức năng tạo ra các truy vấn toàn cục cho mô hình 
# Các truy vấn toàn cục này được sử dụng để thu thập thông tin từ toàn bộ hình ảnh hoặc chuỗi đầu vào 
# Thay vì tập chung vào một phần nhỏ hoặc một vùng cụ thể 

# Trong GC ViT các truy vấn toàn cục này có thể giúp mô hình hiểu được bối cảnh rộng lớn hơn và mỗí quan
# hệ giữa các phần khác nhau của dữ liệu đầu vào 
class GlobalQueryGenerator(layers.Layer):
    """Global query generator.

    Args:
        keepdims: to keep the dimension of FeatureExtraction layer.
        For instance, repeating log(56/7) = 3 blocks, with input
        window dimension 56 and output window dimension 7 at down-sampling
        ratio 2. Please check Fig.5 of GC ViT paper for details.
    """
    def __init__(self, keepdims=False, **kwargs):
        super().__init__()
        self.keepdims = keepdims

    # Thiết lập phương thức Build để tạo 1 danh sách chứa các lớp 
    # featuresExtract được đặt tên theo chỉ số của chúng và giá trị keepdims
    def build (self, input_shape):
        # self.to_q_global là danh sách chưa các lớp FeaturesExtract 
        # mỗi phần tử trong đó đựa Lấy theo chỉ số của keepdims [True, false, True]
        # tức là với keepdims thì enumerate sẽ lặp 3 lần 
        self.to_q_golbal = [
            FeatureExtraction(keepdims, name=f"to_q_global_{i}")
            for i, keepdims in enumerate(self.keepdims)
        ]
        # Cuối cùng cho nó có thể kế thừa lại phương thức này
        super().build(input_shape)
    
    # Thiết lập phương thức call gọi và xử lý các lớp 
    def call (self, inputs , **kwargs):
        x = inputs 
        # Duyệtq au danh sách các lớp trong list to_q_global
        for layer in self.to_q_golbal:
            # Áp dụng mỗi lớp FeaturesExtract lên dữ liệu đầu vào x
            x = layer(x)
        return x 
    


Window Attention

In [ ]:
# Xây dựng khối Windown Attention đây là một kiến trúc xử lý
# của mô hình Swin transformer mục tiêu áp dụng cửa sổ chú ý lên các khung ảnh 
# Tăng cường mối liên kết các pixel trong hình ảnh cũng như khả năng khái quát hóa mô hình 
class WindowAttention(layers.Layer):
    """Local window attention.

    This implementation was proposed by
    [Liu et al., 2021](https://arxiv.org/abs/2103.14030) in SwinTransformer.

    Args:
        window_size: window size.
        num_heads: number of attention head.
        global_query: if the input contains global_query
        qkv_bias: bool argument for query, key, value learnable bias.
        qk_scale: bool argument to scaling query, key.
        attention_dropout: attention dropout rate.
        projection_dropout: output dropout rate.
    """
    # Thiết lập phương thức khởi tạo và định nghĩa các tham số cho mô hình 
    def __init__(
        self,
        window_size,
        num_heads,
        global_query,
        qkv_bias=True,
        qk_scale=None,
        attention_dropout=0.0,
        projection_dropout=0.0,
        **kwargs,
    ):
        super().__init__(**kwargs)
        # định nghĩa các tham số 
        self.window_size = window_size
        self.num_heads = num_heads
        self.global_query = global_query
        self.qkv_bias = qkv_bias
        self.qk_sclae = qk_scale
        # Định nghĩa tham số rời bỏ 
        self.attention_dropout = attention_dropout
        self.projection_dropout = projection_dropout

    # Thiết lập phương thức Build để xây dựng các lớp cấu trúc xử lý 
    def build (self, input_shape):
        # Lấy ra chiều cuối cùng của tenspr input 
        embed_dim = input_shape[0][-1] # bỏ đi chiều thứ nhất 
        # Tính toán số chiều cho mỗi đầu chú ý = embed_dim // num_heads
        head_dim = embed_dim // self.num_heads
        # Tính toán tham số scale cho attention 
        self.scale = self.qk_sclae or head_dim**-0.5
        # Tính toán qkv size = 3 - self.global_query kết quả là 1 giá trị >= 2 <= 3
        # được sử dụng để định danh số lượng vector truy vấn q k v
        self.qkv_size = 3 - int(self.global_query)
        # Khởi tạo QKV attention bằng một lớp Dense 
        self.qkv = layers.Dense(units = embed_dim* self.qkv_size,
                use_bias= self.qkv_bias , name="qkv")
        
        # Khởi tạo một bảng vị trí tương đối và các giá trị của nó mục đích của việc này 
        # cho mỗi cặp cửa sổ trong các phần tử shape = num_window_element , num_heads
        self.realtive_position_bias_table = self.add_weight(
            name = "related_position_bias_tabel",
            # shape =  2 chiều cửa sổ * 2 là số lượng phần tử trong 1 hình ảnh 
            shape = [
                (2* self.window_size[0] - 1) * (2 * self.window_size[1] -1),
            self.num_heads ,],
            # Sau đó khởi tạo các trọng số cho bảng vị trí tươnh đối 
            # bằng phương pháp khởi tạo trọng số phân phối chuẩn cắt ngọn 
            # được giới hạn trong một khoảng nhất định
            initializer=keras.initializers.TruncatedNormal(stddev=0.2),
            # Đặt các tham số của bảng này có thể được cập nhật trong quán trình daod tạo 
            trainable = True ,
            dtype = self.dtype , 
        )
        # Thêm các lớp Xử lý cẩn thiết 
        self.attn_drop = layers.Dropout(self.attention_dropout, name="attn_drop")
        self.proj = layers.Dense(embed_dim, name="proj")
        self.proj_drop = layers.Dropout(self.projection_dropout, name="proj_drop")
        # Cuối cùng là hàm kích hoạt Activation funcion
        self.softmax = layers.Activation("softmax", name="softmax")
        super().build(input_shape)

    # Thiết lập phương thức Xây dựng vị trí tương đối và giá trị tương ứng 
    def get_related_position_index(self):
        # Lấy ra 2 chiều của cửa sổ 
        coords_h = np.arange(self.window_size[0])
        # width 
        coords_w = np.arange(self.window_size[1])
        # Xây dựng ma trận có kích thưov h , w 
        # đặt indexing = ij để mỗi phần tủ trong ma trận là duy nhất
        coords_matrix = np.meshgrid(coords_h , coords_w , indexing='ij')
        # Xây dựng tensor corrd bằng cách xếp chồng 2 ma trận coords lên nhau 
        # kết quả là 1 tensor shape = [2 , w , h]
        coords = np.stack(coords_matrix)
        # sau đó làm phẳng lại tensor này với shape [2 , -1] với tham số 
        # - 1 có nghĩa là ở đây tự tính = num_element 
        coords_flatten = coords.reshape(coords ,[2 -1])
        # Xây dựng ma trận relative_coords  bằng cách thêm chiều cho ma trận flattent
        # sau đó thực hiện phép trừ để có được ma trận relative_coords 
        # shape = [2, num_window_elements, num_window_elements] 
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :,]
        # Sau đó chuyển vị các chiều của ma trận này cho nhau [1 , 2 ,0]
        # shape = [num_element , num_element , 2] chiều cuối cùng cho biết khoảng cách vị trí 
        # hàng và cột 
        relative_coords = relative_coords.transpose([1 , 2 , 0])
        # Cộng các chỉ số hàng và cột của ma trận với 1 hằng số duy nhất 
        relative_coords_xx = relative_coords[:, :, 0] + self.window_size[0] - 1
        relative_coords_yy = relative_coords[:, :, 1] + self.window_size[1] - 1
        # Sau đó ta nhân tọa độ x tương đối của ma trận với hằng số để chuyển đổi tọa độ x tương đối
        # thành một giá trị có thêt kết hợp với toạn độ y tương đối 
        # để tạo ra một chỉ số duy nhất cho mỗi vị trí 
        relative_coords_xx = relative_coords_xx * (2 * self.window_size[1] - 1)
        # Cuối cùng cộng tọa độ x và y tương đối đã được chỉnh để tạo một chỉ số vị trí tương đối duy nhất 
        # cho mỗi điểm trong cửa sổ 
        relative_position_index = relative_coords_xx + relative_coords_yy 
        # Cuối cùng trả về ma trận vị trí tương đối 
        return relative_position_index # shape = [num_elemnet , num_element]
    
    # Xây dựng phương thức tính toán Attention , ma trận possition bias
    def call (self, inputs, **kwargs):
        # kiểm tra xem giá trị Golobal_query có tồn tại
        if self.global_query:
            # gán giá trị 2 biến inputs và q_global = Inputs 
            # input là 1 tensor shape [batch_size , h*w , c]
            inputs , q_global = inputs 
            # Lấy ra kích thước Lô batch_size 
            B = tf.shape(q_global)[0]
        # Trường hợp còn lại 
        else : 
            # Gán cho inputs = batch_size 
            inputs = inputs[0]
        
        # Lấy ra kích thước của tensor đầu vào shape = [batch_size * Num_window , num_tokens , channels]
            # Note channel trong swin transformer = embedim 
        B_, N , C = inputs.shape 
        # Tạo tensor QKV shape = [Batch_size , num_heads ,size ,3 * head_dim]
        qkv = self.qkv(inputs)
        # Định hình lại tensor với shape  =[B_, size, self.qkv_size(3), self.num_heads, C // self.num_heads]
        qkv = tf.reshape(
            qkv, [B_, N, self.qkv_size, self.num_heads, C // self.num_heads]
        )
        # Chuyển vị tensor qkv thành hình dạng shape = [3 , batch_size , num_heads ,size , embed_dim ] 
        # mục đích để có thể tách chiều tensor qkv thành các tensor riêng lẻ 
        qkv = tf.transpose(qkv, [2, 0, 3, 1, 4])
        # Kiểm tra xem giá trị biếnn global_query có tồn tại không 
        if self.global_query:
            # Nếu có tách 2 tensor q và k từ tensor qkv thành 2 phần theo chiều đầu tiên 
            k, v = tf.split(
                qkv, indices_or_sections=2, axis=0
            )  # for unknown shame num=None will throw error

            # Lặp lại q_global (là giá trị đầu vào với số lần bằng với số lượng cửa sổ) để mỗi cứa sổ 
            # trên 1 hình ảnh có cùng một bản sao của gLOBAL .
            q_global = tf.repeat(
                q_global, repeats=B_ // B, axis=0
            )  # num_windows = B_//B => q_global same for all windows in a img

            # Định hình lại kích thước tensor q = [Num_window , size , num_head , embed_dim]
            q = tf.reshape(
                q_global, new_shape=[B_, N, self.num_heads, C // self.num_heads]
            )
            q = tf.transpose(q, axes=[0, 2, 1, 3])
        else:
            # Tách qkv thành 3 vectoe riêng biệt chia làm 3 phần theo chiều đầu tiên 
            q, k, v = tf.split(qkv, indices_or_sections=3, axis=0)
            # Xóa bỏ đi chiều kích thước = 1 axis= 0 nếu chiều đầu = 1 sẽ loại bỏ 
            q = tf.squeeze(q, axis=0)

        # Tuowng tự như tensor k và v
        k = tf.squeeze(k, axis=0)
        v = tf.squeeze(v, axis=0)

        # Nhân Vector q với 1 tham số tỷ lệ scale 
        q = q * self.scale
        # Tính toán attention q*K.T kết quả 1 tensor mới shape = [batch_size ,  num_heads , size ,size]
        # là kết quả sau khi @ K.T shape = [batch_size , num_head ,head_dim , size]
        attn = q@ tf.transpose(k , perm=[0, 1, 3 ,2])
        # Tinhs toans ma trận bias vị trí tương đối 
        # Sử dụng tf.gather để trích xuất hình dạng cho ma trận relative_position_bias 
        # Nhằm mục đích lấy các giá trị từ ma trận relative_position_bias_tabel theo chỉ số của ma trận relative_position_index 
        relative_position_bias = tf.gather(
            self.realtive_position_bias_table,
            # Thay đổi hình dạng của ma trận relative_position_index thành ma trận 1 chiều 
            # với số lượng các tham số tự tính 
            tf.reshape(self.get_relative_position_index(), new_shape=[-1]),
        ) # shape = [num_heads , num_window_element * num_window_elemnet]

        # Sau đó ta resahpe lại tensor relative_position_bias shape = [ num_window_element , num_window_element , num_head]
        relative_position_bias = tf.reshape(
            relative_position_bias , 
            shape = [
                # shape = [ num_window_element , num_window_element , num_head]
                self.window_size[0] * self.window_size[1],
                self.window_size[0] *  self.window_size[1],
                -1
            ],
        )
        # Sau đó đảo chiều ma trận này 
        relative_position_bias = tf.transpose(relative_position_bias , perm=(2 ,0 ,1))
        # Rồi cộng ma trận Attention với am trận này để thêm bias cho vị trí tương đối 
        # trong cửa sổ chú ý  // 2 ma trận này có cùng kích thước nên S sẽ có dạng [batch_size , num_heads, size , size]
        attn = attn + relative_position_bias[None,]
        # Chuyển đến hàm softmax 
        attn = self.softmax(attn)
        # Qua 1 lớp tách rơi dropout 
        attn = self.attn_drop(attn)

        # Tính điểm socre cho attention shap = [ batch_size , num_heads , size , head_dim]
        x_qkv = attn @ v
        # Hoán vị các chiều của x_qkv để có dạng (batch_size, size, num_heads, head_dim)
        x_qkv = tf.transpose(x_qkv, perm=(0, 2, 1, 3))
        # Thay đổi hình dạng của x_qkv thành (batch_size, size, channels), 
        # để nối các đầu chú ý lại với nhau theo chiều thứ 3 
        x_qkv = tf.reshape(x_qkv, shape=(B_, N , C))
        # áp dụng lớp dense để biến đổi về 1 tensor có kích thước ban đầu và 1 lớp bỏ học 
        x_qkv = self.proj(x_qkv)
        # Nhúng tuyến tính và qua 1 lớp tách rời 
        x_qkv = self.proj_drop(self.proj(x_qkv))
        return x_qkv




Helper functions

In [ ]:
# Xây dựng hàm Dropath để loại bỏ ngẫu nhiên cho một tensor đầu vào 
layers.Dropout(0.1) 
class DropPath(layers.Layer):
    def __init__(self, drop_prob=None , **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob 

    def call(self, x):
        # lấy ra kích thước của tensor đầu vào x 
        input_shape = tf.shape(x)
        # lấy ra kích thước lô 
        batch_size = input_shape[0]
        # lấy ra số chiều của x bằng hàm rank 
        rank = x.shape.rank #  = 4 shape x [batch_size , window_size , widow_size , channels]
        # tạo 1 biến shape có shape = batch_size , 1 , 1 ,1 
        # đầu tiên ta tạo ma biến typle với batch_size phàn tử là chiều đầu tiên của shape 
        # sau đó ta tính toán số chiều còn lại  = 1 *(rank-1) tức là 3 chiều 
        # với shape 3 chiều  = 1 
        shape = ( batch_size,) + (1,) * (rank-1) # shape = [batch_size , 1 , 1 ,1]
        # sau đó tạo 1 tensor ngẫu nhiên = xác xuất 1 - drop_prob  + shape 
        # mục đích tạo ra 1 tensor với các phần tử đc lấy ngẫu nhiên [0 -> 1]
        random_tensor = (1 - self.drop_prob) + tf.random.uniform(shape , dtype=x.dtype)
        # Xây dựng một ma trận Path_mask bằng cách làm tròn xuống các tỷ lệ của tensor 
        path_mask = tf.floor(random_tensor)
        # sau đó tính đầu ra bằng thực hiện chia cho tỷ lệ 1 - drop_prob rồi nhân với ma trận 
        # tỷ lệ path_mask 
        output = tf.math.divide(x , 1 - self.drop_prob) * path_mask
        return output 
    


Swin Block

In [ ]:
class Block(layers.Layer):
    """"
        GCVIT block. 

        Args : 
            window_size : window_size . 
            num_heads : number of attention head 
            global_query : apply global window attention
            mlp_rotio : MLP rotio
            qkv_bias: bool argument for query, key, value learnable bias.
            qk_scale: bool argument to scaling query, key.
            drop: dropout rate.
            attention_dropout: attention dropout rate.
            path_drop: drop path rate.
            activation: activation function.
            layer_scale: layer scaling coefficient.
    """

    # Thiiết lập phương thức khởi tạo và định nghĩa các tham số 
    def __init__(
        self, window_size, num_heads,
        global_query, mlp_ratio=4.0, qkv_bias=True,
        qk_scale=None, dropout=0.0, attention_dropout=0.0,
        path_drop=0.0, activation="gelu", layer_scale=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.window_size = window_size
        self.num_heads = num_heads
        self.global_query = global_query
        self.mlp_rotio = mlp_ratio
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.dropout = dropout
        self.attention_dropout = attention_dropout
        self.path_drop = path_drop
        self.activation = activation
        self.layer_scale = layer_scale

    # Xây dựng khối xử lý các lớp chức năng 
    def build (self, input_shape):
        # Lấy ra các kích thước của tensor đầu vào '
        B , H , W , C = input_shape[0]
        # Xây dựng 2 lớp chuẩn hóa layernormal 
        self.norm1 = layers.LayerNormalization(-1 , 1e-05, name="norm1")
        # Xây dựng Cửa sổ chú Ý Windown Attention 
        self.attn = WindowAttention(
            window_size= self.window_size,
            num_heads=self.num_heads,
            global_query=self.global_query,
            qkv_bias=self.qkv_bias,
            qk_scale=self.qk_scale,
            attention_dropout=self.attention_dropout,
            projection_dropout=self.dropout,
            name="attn",
        )
        # Sau đó xây dựng 2 lớp tách rời dropout_path  
        self.drop_path1 = DropPath(self.path_drop)
        self.drop_path2 = DropPath(self.path_drop)
        self.norm2 = layers.LayerNormalization(-1 , 1e-05, name="norm1")

        # Xây dựng khối xử lý MLP 
        self.mlp = MLP(
            hidden_features=int(C * self.mlp_rotio),
            dropout= self.dropout, 
            activation= self.activation, name="mlp",
        )
        # Kiểm tra xem giá trị self.layer_scale có tồn tại hay không 
        if self.layer_scale is not None: 
            # Nếu tồn tại định nghĩa 2 biến gamma là một vector biểu diễn các trọng 
            # số được khởi tạo với tỷ lệ self.layer_scale 
            self.gamma1 = self.add_weight(
                name="gamma1", shape= [C],
                initializer= keras.initializers.Constant(self.layer_scale),
                # đặt các tham số này có thể được cập nhật khi huấn luyện
                trainable = True , dtype = self.dtype , 
            )
            # Tương tự định nghĩa thêm giá trị gamma2 
            self.gamma2 = self.add_weight(
                name="gamma2",
                shape=[C],
                initializer=keras.initializers.Constant(self.layer_scale),
                trainable=True,
                dtype=self.dtype,
            )
        # Trường hợp còn lại tức không tồn tại giá trị layer_scale 
        else:
            # Gán trực tiếp gamma1 và gamma2 = 1.0 
            self.gamma1 = 1.0
            self.gamma2 = 1.0
        # Tiếp theo ta tính toán số lượng cửa sổ trên 1 hình ảnh 
        # Bằng cách nhân thương của 2 chiều H và W của hình ảnh với kích thước của cửa sổ cho nhau 
        self.num_windows = int(H // self.window_size) * int(W // self.window_size)
        # Cuối cùng ta đặt phương thức Build này ở chế độ kế thừa để có thể tái sử dụng 
        # lại các cấu trúc 
        super().build(input_shape)

    # Thiết lập phương thức call để thực hiện xử lý các lớp cấu trúc trong khối 
    def call(self, inputs, **kwargs):
        # Đầu tiên kiểm tra xem giá trị global_Quẻy 
        if self.global_query:
            # Gán giá trị cho 2 biến inputs và q_global = inputs (là 1 tuples gồm 2 phần tử)
            inputs , q_global = inputs 
        # else inputs = inputs [0]
        else:
            inputs = inputs[0]
        B, H, W, C = tf.shape(inputs)
        x = self.norm1(inputs)
        # create windows and concat them in batch axis
        x = self.window_partition(x, self.window_size)  # (B_, win_h, win_w, C)
        # flatten patch
        x = tf.reshape(x, new_shape=[-1, self.window_size * self.window_size, C])
        # attention
        if self.global_query:
            x = self.attn([x, q_global])
        else:
            x = self.attn([x])
        # reverse window partition
        x = self.window_reverse(x, self.window_size, H, W, C)
        # FFN
        x = inputs + self.drop_path1(x * self.gamma1)
        x = x + self.drop_path2(self.gamma2 * self.mlp(self.norm2(x)))
        return x
    
    # Xây dựng phương thức xử lý tạo vách ngăn cửa sổ WIndow Attention 
    def window_partition(self, x, window_size):
        """
        Args:
            x: (B, H, W, C)
            window_size: window size
        Returns:
            local window features (num_windows*B, window_size, window_size, C)
        """
        # Lấy ra kích thước của tensor đầu vào [batch_size, h , w , channels]
        B, H, W, C = tf.shape(x)
        # Reshape lại x thành hình dạng [-1 , patch_num_H , window_size,
        # patch_num_W , window_size , Channle] với tham số  -1 
        # là số lượng hình ảnh trên 1 cửa sổ 
        x = tf.reshape(
            x,
            new_shape=[
                -1,
                H // window_size, # số lượng ảnh theo chiều dọc
                window_size, 
                W // window_size, # Số lượng ảnh theo chiều ngang 
                window_size,
                C,
            ],
        )

        # Reshape lại hình dạng của tensor shape = [-1 , patch_num_y , patch_num_x , window_size , window_size , channels]
        x = tf.transpose(x, perm=[0, 1, 3, 2, 4, 5])
        # Định dạng lại kích thước của cửa sổ và trả nó về shape 
        # = [-1 window_size , window_size, Channel] -1 là số lượng cửa sổ
        windows = tf.reshape(x, new_shape=[-1, window_size, window_size, C])
        # Trả về kết quả là mộ tensor chứa số lượng cửa sổ kích thước và kênh màu 
        return windows
    # Xây dượng phương thức cứa ổ chuyển đổi phương thức này sẽ thược hiện ngược lại 
    # so với phương thức trên 
    def window_reverse(self, windows, window_size, H, W, C):
        """
        Args:
            windows: local window features (num_windows*B, window_size, window_size, C)
            window_size: Window size
            H: Height of image
            W: Width of image
            C: Channel of image
        Returns:
            x: (B, H, W, C)
        """
        # Định dạng lại x về dạng [-1 ,num_patch_H , num_patch_W ,size , size ,channels ]
        x = tf.reshape(
            windows,
            new_shape=[
                -1,
                H // window_size, # Tính toán num_patch H
                W // window_size, # Tính toán num_batch w
                window_size,
                window_size,
                C,
            ],
        )
        # chuyển đổi tensor về shape = [number , patch_num_y , patch_num_x , size_window , size_window ,c]
        x = tf.transpose(x, perm=[0, 1, 3, 2, 4, 5])
        # định hình lại kích thước cho x với -  1 là tham số tự tính cho phù hợp 
        # với bước tính toán để có được số lượng cửa sổ phù hợp 
        x = tf.reshape(x, new_shape=[-1, H, W, C])
        return x 

Level Block

In [ ]:
# Xây dựng lớp Leval để xử lý các tác vụ liên quan đến cấu hình accs tham số .. trong GCViT 
class Level(layers.Layer):
    """GCViT level.

    Args:
        depth: number of layers in each stage.
        num_heads: number of heads in each stage.
        window_size: window size in each stage.
        keepdims: dims to keep in FeatureExtraction.
        downsample: bool argument for down-sampling.
        mlp_ratio: MLP ratio.
        qkv_bias: bool argument for query, key, value learnable bias.
        qk_scale: bool argument to scaling query, key.
        drop: dropout rate.
        attention_dropout: attention dropout rate.
        path_drop: drop path rate.
        layer_scale: layer scaling coefficient.
    """
    # Xây dựng phương thức khởi tạo và định ghĩa các tham số 
    def __init__(
        self, depth,  num_heads,
        window_size,  keepdims,
        downsample=True,  mlp_ratio=4.0,
        qkv_bias=True, qk_scale=None,
        dropout=0.0,  attention_dropout=0.0,
        path_drop=0.0, layer_scale=None,
        **kwargs,
    ):
        # Định nghĩa các tham số 
        super().__init__(**kwargs)
        self.depth = depth
        self.num_heads = num_heads
        self.window_size = window_size
        self.keepdims = keepdims
        self.downsample = downsample
        self.mlp_ratio = mlp_ratio
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.dropout = dropout
        self.attention_dropout = attention_dropout
        self.path_drop = path_drop
        self.layer_scale = layer_scale

    def build(self, input_shape):
        # kiểm tra xem pathdrop có phải 1 dnah sách hay không . Nếu không nó sẽ tạo một danh sách mới với 
        # Phần tử duy nhất là self.path_drop và nhân lặp nó với self.depth lần 
        path_drop = (
            # [self.path_drop] tạo 1 danh sách mới với 1 phần tử duy nhất là giá trị của self.path_drop 
            [self.path_drop] * self.depth # Nhân lặp danh sách với self.depth lần để tạo 1 danh sách mới với depth phần tử 
            # kiểm tra xem self.path drop có phải 1 danh sách list 
            if not isinstance(self.path_drop, list)
            # Nếu nó đã là 1 danh scahs list đoạn mã sẽ không thực thi gì và giữ nguyên giá trị của self.path_drop 
            else self.path_drop
        )
        # Gọi đến lớp xử lý Windown attention là kiến trúc cốt lõi của mô hình 
        self.blocks = [
            Block(
                # Truyền vào khôi block các tham số 
                window_size=self.window_size,
                num_heads=self.num_heads,
                global_query=bool(i % 2),
                mlp_ratio=self.mlp_ratio,
                qkv_bias=self.qkv_bias,
                qk_scale=self.qk_scale,
                dropout=self.dropout,
                attention_dropout=self.attention_dropout,
                path_drop=path_drop[i],
                layer_scale=self.layer_scale,
                name=f"blocks_{i}",
            )
            for i in range(self.depth)
        ]
        # Giảm số chiều không gia mẫu 
        self.down = ReduceSize(keepdims=False, name="downsample")
        # thực hiện truy vấn toàn cục để tăng cường khả năng biểu diễn không gian mẫu 
        self.q_global_gen = GlobalQueryGenerator(self.keepdims, name="q_global_gen")
        super().build(input_shape)

    def call(self, inputs, **kwargs):
        x = inputs
        q_global = self.q_global_gen(x)  # shape: (B, win_size, win_size, C)
        for i, blk in enumerate(self.blocks):
            if i % 2:
                x = blk([x, q_global])  # shape: (B, H, W, C)
            else:
                x = blk([x])  # shape: (B, H, W, C)
        if self.downsample:
            x = self.down(x)  # shape: (B, H//2, W//2, 2*C)
        return x

Model 

In [ ]:
class GCViT(keras.Model):
    """"
        GCViT model. 

    Args: 
        window_size : window size in each stage.
        embed_dim: feature size dimension.
        depths: number of layers in each stage.
        num_heads: number of heads in each stage.
        drop_rate: dropout rate.
        mlp_ratio: MLP ratio.
        qkv_bias: bool argument for query, key, value learnable bias.
        qk_scale: bool argument to scaling query, key.
        attention_dropout: attention dropout rate.
        path_drop: drop path rate.
        layer_scale: layer scaling coefficient.
        num_classes: number of classes.
        head_activation: activation function for head.
    """
    # Thiết lập phương thức khởi tạo và định nghĩa các tham số 
    def __init__(
        self, window_size, embed_dim, depths, num_heads,
        drop_rate=0.0, mlp_ratio=3.0, qkv_bias=True,
        qk_scale=None, attention_dropout=0.0, path_drop=0.1,
        layer_scale=None, num_classes=1000, head_activation="softmax",
        **kwargs,
    ):
        # Định nghĩa các tham số 
        super().__init__(**kwargs)
        self.window_size = window_size # kích thước cửa sổ
        self.embed_dim = embed_dim # Kích thước nhúng
        self.depths = depths # số lớp layer
        self.num_heads = num_heads # Đầu chú ý
        self.drop_rate = drop_rate # dropout_rate 
        self.mlp_ratio = mlp_ratio # MLP 
        self.qkv_bias = qkv_bias # QKV USE BIAS
        self.qk_scale = qk_scale # SCALE FOR QKV ATTENTION
        self.attention_dropout = attention_dropout # Attention dropout 
        self.path_drop = path_drop
        self.layer_scale = layer_scale
        self.num_classes = num_classes 
        self.head_activation = head_activation

        # Nhúng các bản vá hình ảnh 
        self.patch_embed = PatchEmbed(embed_dim=embed_dim, name="patch_embed")
        self.pos_drop = layers.Dropout(drop_rate, name="pos_drop")
        # tạo một mảng numpy path_drop gia giátrij [0.0 -> path_drop] số lượng phần tử 
        # = tổng các phần tử của mảng depths 
        path_drops = np.linspace(0.0 , path_drops, sum(depths))
        # và danh sách tham số keepdims = các giá trị 0 , 1 true or false
        keepdims = [(0, 0, 0), (0, 0), (1,), (1,)]
        # Khơỉ tạo một danh sách levels = []
        #  sẽ được điền bằng các đối tượng GCBlock, là các khối tự chú ý toàn cục và cục bộ.
        self.levels = []
        # Duyệt qua 1 danh sách các phần tử 0 -> độ dài danh sách depths 
        for i in range (len(depths)):
            # Gán giá trị cho biến path_drop bằng độ dài 1 đoạn của mảng path_drop tương ứng 
            # với mỗi mức sau đó sử dụng tolist để chuyển thành 1 danh sách
            # và sử dụng nó để khởi tạo một đối tượng GCBlock 
            path_drop = path_drops[sum(depths[:i])  : sum(depths[: i + 1])].tolist()
            # Và sử dụng để tạo đối tượng GCBlock 
            level = Level(
                depth=depths[i],
                num_heads=num_heads[i],
                window_size=window_size[i],
                keepdims=keepdims[i],
                downsample=(i < len(depths) - 1),
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                dropout=drop_rate,
                attention_dropout=attention_dropout,
                path_drop=path_drop,
                layer_scale=layer_scale,
                name=f"levels_{i}",
            )
            # thêm nó vào danh sách self.levels 
            self.levels.append(level)
        # Thực hiện chuẩn hóa đầu ra 
        self.norm = layers.LayerNormalization(axis=-1 , epsilon=1e-5, name="norm")
        # chuyển qua 1 lớp gộp trung bình  và cuối cùng la 1 lớp dày đặc 
        self.pool = layers.GlobalAvgPool2D(name="pool")
        self.head = layers.Dense(num_classes, name="head", activation=head_activation)

    def build(self, input_shape):
        super().build(input_shape)
        self.built = True

    # Thiết lập phương thức call để gọi lại các lớp xử lý và thực hiện tiến trình xử lý 
    def call(self, inputs, **kwargs):
        # nhúng hình ảnh 
        x = self.patch_embed(inputs)
        # Chuyển qua 1 lớp tách rời
        x = self.pos_drop(x)
        # Duyệt qua danh sách các đối tượng trong danh scahs level 
        for level in self.levels:
            # áp dụng khối xử lý swin lên đầu vào x
            x = level(x)  # shape: (B, H_, W_, C_)
        # thực hiện chuẩn hóa 
        x = self.norm(x)
        x = self.pool(x)  # shape: (B, C__)
        # cuối cùng là lớp phân loại 
        x = self.head(x)
        return x
    
    # Xây dựng kiến trúc mô hình 
    def build_graph(self, input_shape=(224, 224, 3)):
        """
        ref: https://www.kaggle.com/code/ipythonx/tf-hybrid-efficientnet-swin-transformer-gradcam
        """
        # Thiết lập lớp đầu vào 
        x = keras.Input(shape=input_shape)
        # Thiết lập mô hình GCVit 
        return keras.Model(inputs=[x], outputs=self.call(x), name=self.name)

    # Tóm tắt kiến trúc mô hình và tham số tôngt quát 
    def summary(self, input_shape=(224, 224, 3)):
        return self.build_graph(input_shape).summary()



Build Model

In [ ]:
# Model Configs
config = {
    "window_size": (7, 7, 14, 7),
    "embed_dim": 64,
    "depths": (2, 2, 6, 2),
    "num_heads": (2, 4, 8, 16),
    "mlp_ratio": 3.0,
    "path_drop": 0.2,
}
ckpt_link = (
    "https://github.com/awsaf49/gcvit-tf/releases/download/v1.1.6/gcvitxxtiny.keras"
)

# Build Model chuyền tham số cấu hình qua giá trị kwargs 
model = GCViT(**config)
# Định cấu hình co ảnh đầu vào shape 244 * 244 * 3 
inp = np.array(np.random.uniform(size=(1, 224, 224, 3)))
out = model(inp)

# Load Weights Tải trọng số của mô hình đã đựoc đào tạo trước 
ckpt_path = keras.utils.get_file(ckpt_link.split("/")[-1], ckpt_link)
model.load_weights(ckpt_path)

# Summary 
model.summary((224, 224, 3))

Sanity check for Pre-Trained Weights

In [ ]:
# Thiết lập phương thức tiền xử lý hình ảnh trong keras với tập dữ liệu chelsea 
img = keras.applications.imagenet_utils.preprocess_input(
    chelsea() , mode = "torch"
) # Chelsea the cat 
# Resize hình ảnh về dạng tiêu chuẩn 224 * 224 * 3 và khởi tạo các batch
img = tf.image.resize(img , (224, 224))[None, ]
# Thử nghiệm mô hình với tiền đào tạo trước hình ảnh và dự đoán hình ảnh 
pred = model(img)
# Lấy ra xác xuất dự đoán cho hình ảnh 
pred_dec = keras.applications.imagenet_utils.decode_predictions(pred)[0]

# Hiển thị hình ảnh và in ra các thông số dự đoán cho mô hình 
print("\n# Image:")
# Định cấu hình khung hiển thị hình ảnh 6 *6  inch 
plt.figure(figsize=(6, 6))
# Sử dụng plt.imshow để hiển thị ra hình ảnh
plt.imshow(chelsea())
plt.show()

# In ra Top 5 phân loại cho hình ảnh có xác xuất cao nhất 
print("# Prediction (Top 5)")
for i in range(5):
    print ("{:<12} : {:0.2f}").format(pred_dec[i][1], pred_dec[i][2])

# Là ngành khoa học kỹ thuật nghiên cứu chế tạo máy thông minh mục đích để học thực hiện các hành vi thông minh 


Fine-tune GCViT Model

In [ ]:
# Định cấu hình các tham số 
IMAGE_SIZE = (224 , 224)

# Hyperparameter 
BATCH_SIZE = 32 
EPOCHS = 5  

# Data class 
CLASSES = [
    "danelion",
    "daisy", 
    "tulips",
    "sùnlowers",
    "reses",
] 

# Độ lệch chuẩn và bình quaan tối thiểu mất mát cho hình ảnh 
MEAN = 255 * np.array([0.485 , 0.456 , 0.406], dtype="float32")
STD = 255 * np.array([0.229, 0.224, 0.225], dtype="float32")
# XÂY dựng tf.data để cho phép mô hình có khả năng tự đồng tùy chỉnh các thông số dữ liệu 
# cho phù hợp với thiết bị 
AUTO = tf.data.AUTOTUNE

Data Loader

In [ ]:
def make_dataset(dataset: tf.data.Dataset, train: bool , image_size: int = IMAGE_SIZE):
    def preprocess(image, label):
        # For training , do augmentation 
        # Kiểm tra xem mô hình có đang trong chế độ huấn luyện không 
        if train : 
            # Kiểm tra xem giá trị xác xuất ngẫu nhiên có lớn hơn 0.5 
            if tf.random.uniform(shape=[]) > 0.5:
                # Nếu thỏa mãn các điều kiện trên thì ta áp dụng các phép tăng cường dữ liệu 
                image = tf.iamge.flip_left_right(image)
            # sau dó ressize lại hình ảnh về kích thước tiêu chuẩn đồng nhất 
            image = tf.image.resize(image, size=image_size , method="bicubic")
            # Biến đổi hình ảnh 
            image (image - MEAN) / STD # normalization 
            return image, label

    if train : 
        # TRỘN DỮ LIỆU VỚI KÍCH THƯỚC BATCH_SIZE * 10 
        dataset = dataset.shuffle(BATCH_SIZE * 10)
    # Áp dụng hàm biến đổi map cho tập dữ liệu
    return dataset.map(preprocess, AUTO).batch(BATCH_SIZE).prefetch(AUTO)

Flower Dataset

In [ ]:
train_dataset , val_dataset = tfds.load(
    "tf_flowers", 
    split = ["train[:90%]" ,"train[:90%]"],
    as_supervised=True , 
    try_gcs=False , # GCS_path is necessary for tpu 
)

# Tạo2 bộ dữ liệu đào tạo và xác thực sử dụng 90 % dữ liệu đào tạo 
train_dataset = make_dataset(train_dataset, True)
val_dataset = make_dataset(val_dataset, False)

Re-Build Model for Flower Dataset

In [ ]:
# Re Build the model 
model = GCViT(**config, num_classes=104)
inp = np.array(np.random.uniform(size=(1, 224, 224, 3)))
out = model(inp)

# Load weight 
# checkpath file 
ckpt_path = keras.utils.get_file(ckpt_link.split("/")[-1], ckpt_link)
# tối ưu hóa mô hình bằng cách tái sử dụng lại tham số đào tạo trước 
model.load_weights(ckpt_path, skip_mismatch=True)

# Trình biên dịch mô hình cho qúa trình tinh chỉnh 
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

Training

In [ ]:
history = model.fit(
    train_dataset, validation_data=val_dataset, epochs=EPOCHS, verbose=1
)